# Simple Window

This tutorial shows how to open a Graphical User Interface using the Metavision SDK UI API.

In [ ]:
import os
import numpy as np
import metavision_sdk_ui as mv_ui
import cv2

### Load names of Keys and Buttons



In [ ]:
key_to_name = {}
for name, key in mv_ui.UIKeyEvent.__members__.items():
    key_to_name[key] = name

button_to_name = {}
for name, button in mv_ui.UIMouseButton.__members__.items():
    button_to_name[button] = name

### Display an image and set event-callbacks

In [ ]:
def test_window(img, use_mt):  
    # Don't modify the original input image
    crt_img = img.copy()
    height = img.shape[0]
    width = img.shape[1]
    
    # Grayscale mode
    if len(img.shape) == 2:
        render_mode =  mv_ui.Window.RenderMode.GRAY
        window_title = "Window GRAY"
        cursor_color = 255
        mouse_text_color = 255
        key_text_color = 255
    # Color mode
    elif img.shape[2] == 3:
        render_mode =  mv_ui.Window.RenderMode.BGR
        window_title = "Window BGR"
        cursor_color = (0, 0, 255)
        mouse_text_color = (0, 255, 0)
        key_text_color = (255, 0, 0)
    else:
        print("Incompatible input image's number of channels. Must be either 1 or 3.")
        return
    
    # Instantiate window
    if use_mt:
        w = mv_ui.MTWindow(window_title, img.shape[1], img.shape[0], render_mode)
    else:
        w = mv_ui.Window(window_title, img.shape[1], img.shape[0], render_mode)
        
    with w:
        # Variables used for the callbacks
        last_x = 0
        last_y = 0
        last_mouse_str = ""
        last_key_str = ""
        close_window = False

        # Callback that will be called when the mouse's cursor moves on the current window
        def cursor_cb(x, y):
            nonlocal last_x
            nonlocal last_y
            last_x, last_y = int(x), int(y)

        # Callback that will be called when a mouse's button is pressed
        def mouse_cb(button, action, mods):
            nonlocal last_mouse_str
            if action == mv_ui.UIAction.RELEASE:
                if button in button_to_name:
                    last_mouse_str = button_to_name[button]

        # Callback that will be called when a key is pressed            
        def keyboard_cb(key, scancode, action, mods):
            nonlocal last_key_str
            nonlocal close_window
            if action == mv_ui.UIAction.RELEASE:
                if key in key_to_name:
                    last_key_str = key_to_name[key]

                if key == mv_ui.UIKeyEvent.KEY_ESCAPE:
                    close_window = True

        # Sets the callbacks
        w.set_cursor_pos_callback(cursor_cb)
        w.set_mouse_callback(mouse_cb)
        w.set_keyboard_callback(keyboard_cb)

        # Display window
        while not w.should_close():
            # Dispatch system events to the window
            mv_ui.EventLoop.poll_and_dispatch()

            # Adjust cursor to current window size
            crt_width, crt_height = w.get_size()
            cursor_x = int((last_x * width) / crt_width)
            cursor_y = int((last_y * height) / crt_height)

            # Superimpose callbacks results on the image
            crt_img = img.copy()
            cv2.circle(crt_img, (cursor_x, cursor_y), 15, cursor_color, 2)
            cv2.putText(crt_img, last_mouse_str, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, mouse_text_color, 2)
            cv2.putText(crt_img, last_key_str, (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, key_text_color, 2)

            # Show the image
            if use_mt:
                w.show_async(crt_img)
            else:            
                w.show(crt_img)

            if close_window:
                w.set_close_flag()

#### Grayscale image

In [ ]:
# Launch the cell and interact with the window using the cursor, keys and mouse buttons
img = cv2.imread("./metavision_ui_window_files/prophesee.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
test_window(img, use_mt=True)

#### BGR image

In [ ]:
# Launch the cell and interact with the window using the cursor, keys and mouse buttons
img = cv2.imread("./metavision_ui_window_files/prophesee.jpg")
test_window(img, use_mt=True)